# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#import csv
city_data = "../WeatherPy/output_data/cities.csv"
city_data_df = pd.read_csv(city_data)
city_df = pd.DataFrame(city_data_df)
city_df

,city,latitude,longitude,max temp (F),humidity,cloudiness,windspeed,country,date
0,Flinders,-34.58,150.86,51.01,71,19,1.01,AU,1596145763
1,Barrow,71.29,-156.79,37.40,93,90,5.82,US,1596145763
2,Ushuaia,-54.80,-68.30,31.17,94,56,4.43,AR,1596145764
3,Chumikan,54.72,135.31,53.73,97,100,16.78,RU,1596145764
4,Yellowknife,62.46,-114.35,69.80,49,75,18.34,CA,1596145764
...,...,...,...,...,...,...,...,...,...
546,Nushki,29.55,66.02,83.48,41,4,7.07,PK,1596145888
547,Mujiayingzi,42.12,118.78,72.21,68,100,7.36,CN,1596145646
548,Abu Kamal,34.45,40.92,92.01,15,0,7.27,SY,1596145888
549,Almeirim,-1.52,-52.58,81.46,62,85,1.61,BR,1596145888


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

locations = city_df[["latitude", "longitude"]]
weights = city_df["humidity"]
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Narrow down city list by conditions
ideal_df1 = city_df.loc[(city_df["max temp (F)"] < 90)]
ideal_df2 = ideal_df1.loc[(city_df["max temp (F)"] > 80)]
ideal_df3 = ideal_df2.loc[(city_df["humidity"] < 60)] 
ideal_df4 = ideal_df3.loc[(city_df["windspeed"] < 10)]
ideal_df5 = ideal_df4.loc[(city_df["cloudiness"] < 10)]
ideal_df = ideal_df5.dropna()
ideal_df

,city,latitude,longitude,max temp (F),humidity,cloudiness,windspeed,country,date
79,Aripuanã,-9.17,-60.63,82.27,36,0,1.59,BR,1596145781
200,Kiryat Gat,31.61,34.76,82.00,58,0,0.81,IL,1596145808
246,Santa Isabel Rio Negro,-0.41,-65.02,82.49,59,0,2.15,BR,1596145818
251,Olivenza,38.68,-7.10,86.00,42,0,1.01,ES,1596145775
275,Pimenta Bueno,-11.67,-61.19,83.30,32,0,3.15,BR,1596145569
381,Lewistown,40.60,-77.57,87.80,45,1,5.82,US,1596145848
451,Vilhena,-12.74,-60.15,84.20,39,0,6.93,BR,1596145866
470,Hauterive,46.09,3.45,82.40,42,0,3.36,FR,1596145870
532,Atikokan,48.76,-91.62,82.08,26,0,5.55,CA,1596145885
546,Nushki,29.55,66.02,83.48,41,4,7.07,PK,1596145888


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#Set up column to hold new info
ideal_df["hotel_name"] = ""
ideal_df

,city,latitude,longitude,max temp (F),humidity,cloudiness,windspeed,country,date,hotel_name
79,Aripuanã,-9.17,-60.63,82.27,36,0,1.59,BR,1596145781,
200,Kiryat Gat,31.61,34.76,82.00,58,0,0.81,IL,1596145808,
246,Santa Isabel Rio Negro,-0.41,-65.02,82.49,59,0,2.15,BR,1596145818,
251,Olivenza,38.68,-7.10,86.00,42,0,1.01,ES,1596145775,
275,Pimenta Bueno,-11.67,-61.19,83.30,32,0,3.15,BR,1596145569,
381,Lewistown,40.60,-77.57,87.80,45,1,5.82,US,1596145848,
451,Vilhena,-12.74,-60.15,84.20,39,0,6.93,BR,1596145866,
470,Hauterive,46.09,3.45,82.40,42,0,3.36,FR,1596145870,
532,Atikokan,48.76,-91.62,82.08,26,0,5.55,CA,1596145885,
546,Nushki,29.55,66.02,83.48,41,4,7.07,PK,1596145888,


In [6]:
# Use lat, lng to use places API to find nearest hotel

hotel_name = []

# use iterrows to iterate through pandas dataframe
for index, row in ideal_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    lat = row["latitude"]
    lng = row["longitude"]
    city = ideal_df["city"]

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types": "lodging",
        "key": g_key
    }
    
    
    #get city name from df
    city_name = row["city"]
    
    params["keyword"] = hotel_name
    
    # Run request
    (f"Retrieving Results for index {index}: {city}")
    response = requests.get(base_url, params=params).json()


    # print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
      #print(response.url)
    results = response["results"]

    #print(json.dumps(hotels, indent=4, sort_keys=True))

    try:
        print(f"Closest hotel to {city_name} is {results[0]['name']}.")
        ideal_df.loc[index, "hotel_name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

    print("------------")
    
# 

   

Missing field/result... skipping.
------------
Closest hotel to Kiryat Gat is רגע בנחת.
------------
Closest hotel to Santa Isabel Rio Negro is Hotel GSA.
------------
Closest hotel to Olivenza is Hotel Heredero.
------------
Closest hotel to Pimenta Bueno is Cristalina Palace Hotel.
------------
Closest hotel to Lewistown is Quality Inn & Suites.
------------
Closest hotel to Vilhena is Hotel Mirage.
------------
Closest hotel to Hauterive is Aletti Palace Hotel.
------------
Closest hotel to Atikokan is Camp Quetico.
------------
Closest hotel to Nushki is Mahran motor showroom.
------------


In [7]:
ideal_df

,city,latitude,longitude,max temp (F),humidity,cloudiness,windspeed,country,date,hotel_name
79,Aripuanã,-9.17,-60.63,82.27,36,0,1.59,BR,1596145781,
200,Kiryat Gat,31.61,34.76,82.00,58,0,0.81,IL,1596145808,רגע בנחת
246,Santa Isabel Rio Negro,-0.41,-65.02,82.49,59,0,2.15,BR,1596145818,Hotel GSA
251,Olivenza,38.68,-7.10,86.00,42,0,1.01,ES,1596145775,Hotel Heredero
275,Pimenta Bueno,-11.67,-61.19,83.30,32,0,3.15,BR,1596145569,Cristalina Palace Hotel
381,Lewistown,40.60,-77.57,87.80,45,1,5.82,US,1596145848,Quality Inn & Suites
451,Vilhena,-12.74,-60.15,84.20,39,0,6.93,BR,1596145866,Hotel Mirage
470,Hauterive,46.09,3.45,82.40,42,0,3.36,FR,1596145870,Aletti Palace Hotel
532,Atikokan,48.76,-91.62,82.08,26,0,5.55,CA,1596145885,Camp Quetico
546,Nushki,29.55,66.02,83.48,41,4,7.07,PK,1596145888,Mahran motor showroom


In [8]:
hotel_df = ideal_df.rename(columns={"city": "City", "latitude": "Lat", "longitude": "Lng", 
                                    "max temp(F)": "Max Temp (F)", "humidity": "Humidity", "cloudiness": "Cloudiness (%)",
                                    "windspeed": "Wind Speed (mph)", "country": "Country", "date": "Date",
                                    "hotel_name": "Hotel Name"})
hotel_df

,City,Lat,Lng,max temp (F),Humidity,Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
79,Aripuanã,-9.17,-60.63,82.27,36,0,1.59,BR,1596145781,
200,Kiryat Gat,31.61,34.76,82.00,58,0,0.81,IL,1596145808,רגע בנחת
246,Santa Isabel Rio Negro,-0.41,-65.02,82.49,59,0,2.15,BR,1596145818,Hotel GSA
251,Olivenza,38.68,-7.10,86.00,42,0,1.01,ES,1596145775,Hotel Heredero
275,Pimenta Bueno,-11.67,-61.19,83.30,32,0,3.15,BR,1596145569,Cristalina Palace Hotel
381,Lewistown,40.60,-77.57,87.80,45,1,5.82,US,1596145848,Quality Inn & Suites
451,Vilhena,-12.74,-60.15,84.20,39,0,6.93,BR,1596145866,Hotel Mirage
470,Hauterive,46.09,3.45,82.40,42,0,3.36,FR,1596145870,Aletti Palace Hotel
532,Atikokan,48.76,-91.62,82.08,26,0,5.55,CA,1596145885,Camp Quetico
546,Nushki,29.55,66.02,83.48,41,4,7.07,PK,1596145888,Mahran motor showroom


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)


# # fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))